In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
features.head()

log_prices = np.log(boston_dataset.target)
log_prices.shape
features.shape
target = pd.DataFrame(log_prices, columns=['PRICE'])
target.shape

(506, 1)

In [3]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

#property_stats = np.ndarray(shape=(1, 11))
#property_stats[0][CRIME_IDX] = features['CRIME'].mean()
#property_stats[0][ZN_IDX] = features['ZN'].mean()
#property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1, 11)


In [4]:
features.head()

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
type(features.mean())

pandas.core.series.Series

In [6]:
type(features.mean().values)

numpy.ndarray

In [7]:
features.mean().values.shape

(11,)

In [8]:
features.mean().values.reshape(1, 11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [12]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)
RMSE

0.18751213519713034

In [35]:
def get_log_estimate(nr_rooms, students_per_classroom,
                     next_to_river=False,
                     high_confidence=True):
  
    #configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    #make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    #Calc Range
    
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [38]:
get_log_estimate(7, 20, next_to_river=True, high_confidence=False)

(3.139449357922929, 3.3269614931200593, 2.9519372227257987, 68)

In [40]:
np.median(boston_dataset.target)

21.2

In [41]:
zillow_median_price = 583.3
scale_factor = zillow_median_price / np.median(boston_dataset.target)
scale_factor

27.514150943396224

In [53]:
log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                               next_to_river=False)
dollar_est= np.e**log_est * 1000 * scale_factor
dollar_est


dollar_hi= np.e**upper * 1000 * scale_factor
dollar_lower= np.e**lower * 1000 * scale_factor

#round the dollar values
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_lower = np.around(dollar_lower, -3)

print(f'The estimated property value is {rounded_est}')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_lower} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0
At 95% confidence the valuation range is
USD 568000.0 at the lower end to USD 1203000.0 at the high end.


In [64]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
        """ Estimate price of property in Boston. 
        
        Keyword arguments:
        rm -- number of rooms in property
        ptratio -- number of students per class room
        chas -- true if the property is next to the river,  false otherwise
        large_range -- true for 95% prediction interval, false for 68%
        """
        
        
        if rm < 1 or ptratio < 1:
            print('That is unrealistic. Try again')
            return
        
        
        log_est, upper, lower, conf = get_log_estimate(nr_rooms=rm,
                                                       students_per_classroom = ptratio,
                                                    next_to_river = chas,
                                                      high_confidence = large_range)
        dollar_est= np.e**log_est * 1000 * scale_factor
        dollar_est


        dollar_hi= np.e**upper * 1000 * scale_factor
        dollar_lower= np.e**lower * 1000 * scale_factor

        #round the dollar values
        rounded_est = np.around(dollar_est, -3)
        rounded_hi = np.around(dollar_hi, -3)
        rounded_lower = np.around(dollar_lower, -3)

        print(f'The estimated property value is {rounded_est}')
        print(f'At {conf}% confidence the valuation range is')
        print(f'USD {rounded_lower} at the lower end to USD {rounded_hi} at the high end.')


In [63]:
get_dollar_estimate(5, 0)

That is unrealistic. Try again


In [65]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.
